In [1]:
allcols=set()
allcolsls = []

In [2]:
import numpy as np
import pandas as pd
import pylab as plt
from tqdm.auto import tqdm
from astropy.io import ascii

In [3]:
import sys
sys.path.append("../")
import sidhelpers

## 1. See how many spectra (%) can be opened by Astropy

In [4]:
# Read spectra metadata
spectra_info = pd.read_csv("../1. download ALL wise data/wiserep_spectra_combined.csv")

# keep p60 only
spectra_info = spectra_info[spectra_info["Telescope"] == "P60"]

In [5]:
spectra_info["Obj. Type"].value_counts()

Obj. Type
SN Ia                4769
SN II                 782
SN Ic                  92
SN Ia-91T-like         90
SN IIn                 62
SN IIb                 54
SN Ib                  53
SN Ic-BL               52
SN IIP                 51
SN Ia-91bg-like        31
TDE                    30
SN Ia-pec              27
SN                     22
CV                     20
Nova                   18
SLSN-II                15
SN Ib/c                15
SN Ibn                 14
SN I                   14
SLSN-I                 13
AGN                    12
SN Icn                 10
SN Iax[02cx-like]       9
SN Ia-CSM               9
Galaxy                  3
SN II-pec               3
QSO                     3
SN Ib-Ca-rich           2
ILRT                    1
SN IIL                  1
LBV                     1
SN Ic-Ca-rich           1
SN Ic-pec               1
Computed-Ia             1
FBOT                    1
SN Ia-SC                1
Name: count, dtype: int64

In [6]:
filenames = spectra_info["Ascii file"].values

In [7]:
# df = ascii.read(f"../1. download ALL wise data/wiserep_data/spectra/{fn}").to_pandas()

In [8]:
# spectra_info["Spec. units"].value_counts().index[0]

In [9]:
# most efficient way to iterate
# https://stackoverflow.com/questions/7837722/what-is-the-most-efficient-way-to-loop-through-dataframes-with-pandas

flux_counter = 0
readerror_counter = 0
errs = []

for fn, wl_unit, spec_unit, flux_ucoeff, lambda_min, lambda_max, del_lambda in tqdm(zip(
    spectra_info["Ascii file"],
    spectra_info["WL Units"],
    spectra_info["Spec. units"],
    spectra_info["Flux Unit Coefficient"],
    spectra_info["Lambda-min"],
    spectra_info["Lambda-max"],
    spectra_info["Del-Lambda"],
), total=len(spectra_info)):
    try:
        df = ascii.read(f"../1. download ALL wise data/wiserep_data/spectra/{fn}").to_pandas()
        allcols.update(df.columns)
        allcolsls = allcolsls + list(df.columns)
    except Exception as ee:
        readerror_counter = readerror_counter + 1
        errs.append([fn,ee])
        continue
    try:
        sidhelpers.check_df(df, wl_unit, spec_unit, lambda_min, lambda_max, del_lambda)
    except Exception as e:
        errs.append([fn,e])
        
        if e=="Flux is not positive!":
            flux_counter=flux_counter+1
        continue

  0%|          | 0/6304 [00:00<?, ?it/s]

In [10]:
flux_counter

0

In [11]:
df_unique = pd.DataFrame({"columns": list(allcols)})
df_unique.to_csv("unique_columns_p60_05april.csv", index=False)
df_unique

,columns
0,obswave
1,FILENAME:
2,col3
3,idk
4,P60
5,col2
6,flam
7,col1
8,flux
9,ZTF18abcezmh_20180703_P60_v1.ascii


In [12]:
readerror_counter

30

In [13]:
errdf = pd.DataFrame(errs, columns=["Ascii file", "error"]).set_index("Ascii file")

In [14]:
errdf.to_csv("errdf_05april.csv")

In [15]:
errdf

,error
Ascii file,
2017gxc_2017-09-30_04-13-40_P60_SEDM_None.txt,Spectral unit conversion from 'Other' not impl...
2017gxc_2017-10-01_04-11-10_P60_SEDM_None.txt,Spectral unit conversion from 'Other' not impl...
iPTF16fcg_2016-09-06_00-00-00.000000_P60_SEDM_iPTF.ascii,Flux is not positive!
SN2016fcg_2016-09-06_11-06-02_P60_SEDM_TNS.ascii,Flux is not positive!
2018cfw_2018-06-10_09-21-07_P60_SEDM_None.txt,Spectral unit conversion from 'Other' not impl...
...,...
SN2025cku_2025-02-27_10-53-27_P60_SEDM_TNS.ascii,Flux is not positive!
SN2025yd_2025-03-05_09-56-24_P60_SEDM_TNS.ascii,Flux is not positive!
SN2025cjg_2025-03-05_05-59-26_P60_SEDM_TNS.ascii,Flux is not positive!


In [18]:
!cat SN2025cku_2025-02-27_10-53-27_P60_SEDM_TNS.ascii	

cat: SN2025cku_2025-02-27_10-53-27_P60_SEDM_TNS.ascii: No such file or directory


In [16]:
errdf[errdf["error"]=="Flux is not positive!"]

,error
Ascii file,


In [17]:
import astropy.units as u
import astropy.constants as const
from astropy.units import equivalencies

# Define the flux in mJy
flux_mJy = 1.0 * u.mJy  # Example value of 1 mJy

# To convert between frequency and wavelength units, we need to specify the wavelength
wavelength = 5500 * u.AA  # Example: optical wavelength at 5500 Angstroms

# Create the equivalency between frequency and wavelength flux densities
# This accounts for the non-linear relationship between frequency and wavelength
fnu_to_flambda = u.spectral_density(wavelength)

# Do the conversion
flux_flambda = flux_mJy.to(u.erg / u.s / u.cm**2 / u.AA, equivalencies=fnu_to_flambda)

print(f"Flux density at {wavelength}:")
print(f"{flux_mJy} = {flux_flambda}")

Flux density at 5500.0 Angstrom:
1.0 mJy = 9.910494479338844e-16 erg / (Angstrom s cm2)
